# 프로젝트 명 : Seoul_Crime

# 1. 프로젝트 개요

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import socket
hostname = socket.gethostname()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. 데이터 개요
### 1) 데이터 읽기

In [7]:
import pandas as pd
import numpy as np
crime_raw_data=pd.DataFrame()

if "Dongmin" in hostname:
    pass
else :
    crime_raw_data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/02.Seoul_Crime/02. crime_in_Seoul.csv",thousands=",",encoding="euc-kr")

crime_raw_data.head()

,구분,죄종,발생검거,건수
0,중부,살인,발생,2.0
1,중부,살인,검거,2.0
2,중부,강도,발생,3.0
3,중부,강도,검거,3.0
4,중부,강간,발생,141.0


### 2) 데이터 개요 보기
- 문제점 :
  - RangeIndex : 65534
  - Not-null 데이터 : 310
  - null 데이터가 많음

In [8]:
crime_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65534 entries, 0 to 65533
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구분      310 non-null    object 
 1   죄종      310 non-null    object 
 2   발생검거    310 non-null    object 
 3   건수      310 non-null    float64
dtypes: float64(1), object(3)
memory usage: 2.0+ MB


### 3) 잘못된 데이터 보기
- 죄종에 어떤 값이 들어가있는지 보기
- null 값에 얼마나 많은 데이터들이 있는지 보기

In [9]:
crime_raw_data["죄종"].unique()

array(['살인', '강도', '강간', '절도', '폭력', nan], dtype=object)

- 310번부터 65534번까지 데이터가 모두 NaN 값임

In [11]:
crime_raw_data[crime_raw_data["죄종"].isnull()]

,구분,죄종,발생검거,건수
310,NaN,NaN,NaN,NaN
311,NaN,NaN,NaN,NaN
312,NaN,NaN,NaN,NaN
313,NaN,NaN,NaN,NaN
314,NaN,NaN,NaN,NaN
...,...,...,...,...
65529,NaN,NaN,NaN,NaN
65530,NaN,NaN,NaN,NaN
65531,NaN,NaN,NaN,NaN
65532,NaN,NaN,NaN,NaN


In [15]:
crime_raw_data=crime_raw_data[crime_raw_data["죄종"].notnull()]

- 새로운 로우 데이터를 보면 정상적으로 보임
  - integer로 보임

In [16]:
crime_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 310 entries, 0 to 309
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구분      310 non-null    object 
 1   죄종      310 non-null    object 
 2   발생검거    310 non-null    object 
 3   건수      310 non-null    float64
dtypes: float64(1), object(3)
memory usage: 12.1+ KB
